<a href="https://colab.research.google.com/github/Koushik-Madhavan/ComparativeAnalysis/blob/main/Jarviss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_groq langchain_core langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [6]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature = 0,
    groq_api_key = "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",
    model_name = "llama-3.1-8b-instant"
)
result = llm.invoke("tell about in one sentence regarding MS.Dhoni fans?")
print(result.content)

MS Dhoni fans, also known as 'Dhoni Army,' are a dedicated and passionate group of cricket enthusiasts who admire and idolize the former Indian cricketer for his exceptional leadership, skill, and sportsmanship.


In [7]:
!pip install pypdf

In [8]:
!pip install chromadb

In [9]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [18]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os

# Step 1: Initialize LLM
def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama3-8b-8192"  # ✅ Valid Groq model
    )
    return llm

# Step 2: Create Vector DB from PDFs
def create_vector_db():
    loader = DirectoryLoader("/content/Data/", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='/content/chroma_db')
    vector_db.persist()

    print("✅ ChromaDB created and data saved")
    return vector_db

# Step 3: Setup Retrieval QA Chain
def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()

    prompt_template = """
    You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    Context: {context}
    User: {question}
    Chatbot:"""

    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )

    return qa_chain

# Step 4: Run Jarviss
def main():
    print("Initializing Jarviss.........")
    llm = initialize_llm()
    db_path = "/content/chroma_db"

    if not os.path.exists(db_path):
        vector_db = create_vector_db()
    else:
        embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
        vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

    qa_chain = setup_qa_chain(vector_db, llm)

    while True:
        query = input("\nKoushik: ")
        if query.lower() == "exit":
            print("Jarviss: Take care of yourself. Goodbye!")
            break
        response = qa_chain.run(query)
        print(f"Jarviss: {response}")

if __name__ == "__main__":
    main()

Initializing Jarviss.........

Koushik: hlo


<ipython-input-18-1043288612>:75: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)


Jarviss: What a thought-provoking question! It's wonderful that you're considering the complexities of mental health and its interconnectedness with our social and physical environments.

The definition of positive mental health you mentioned, which includes a sense of purpose, quality relationships, self-respect, and mastery, is indeed a crucial aspect of overall well-being. However, as you pointed out, individuals don't exist in isolation, and their mental health is influenced by and influences their surroundings.

I'd like to build upon this idea by suggesting that mental health is not just about individual characteristics, but also about the capacity to interact with others and the environment in ways that promote subjective well-being, optimal development, and collective goals consistent with justice.

This more comprehensive definition acknowledges that mental health is not just about individual struggles or triumphs, but also about the social and environmental factors that shape

In [19]:
!pip install gradio